In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "PROJECT_NAME"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [6]:
from langchain import hub
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [8]:
loader = TextLoader("path/to/txt/file")
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [10]:
embedding_fn = SentenceTransformerEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_fn)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

This particular pipeline is trained on a scrape of Marxists.org, a repository of writing by socialist writers. But the local file that is turned into a vector database could be about anything.

In [16]:
rag_chain.invoke("What is anarcho-syndicalism?")

'Anarcho-syndicalism is an intellectual current advocating the abolition of economic monopolies and coercive institutions in society. It proposes a free association of productive forces based on cooperative labor to meet the needs of all members of society. Anarcho-syndicalism combines anarchist socialism with the organizational principles of revolutionary syndicalism to empower workers in shaping economic and social structures.'

In [17]:
rag_chain.invoke("What were Marx's theses on feurbach about?")

"Marx's Theses on Feuerbach were written in 1845 and emphasized the importance of materialism and practical activity in understanding the world. They critiqued traditional philosophy for being passive and abstract, calling for a more active and engaged approach to understanding reality. These theses laid the groundwork for Marx's later development of historical materialism and his critique of capitalism."

In [18]:
rag_chain.invoke("What were Marx's main critiques of the Gotha Program?")

"Marx's main critiques of the Gotha Program included addressing the dictatorship of the proletariat, the transition from capitalism to communism, and the maturity of communism. Engels wrote a foreword to the document when it was first published in 1891, emphasizing its importance in the discussion. Despite public endorsements of the program, Engels remained critical of the unity congress in private correspondence."

In [19]:
rag_chain.invoke("What were Kollontai's contributions?")

"Kollontai's contributions include works on international socialist conferences, women's rights, and anti-war activism. She emphasized the importance of women's emancipation as part of the broader class struggle and criticized suffragette alliances with bourgeois interests. Kollontai's writings and activism reflected her commitment to advocating for the rights of women and workers."

In [20]:
rag_chain.invoke("What did Rosa and Lenin think about the Right to self determination?")

'Rosa Luxemburg believed in freedom for all individuals, regardless of political affiliation, as true freedom only exists when different opinions are allowed. Marx and Engels emphasized the importance of political freedom to achieve economic emancipation for the working class. They saw political revolution in Russia as crucial for social revolution.'

As we can see, the model answers questions about marxists theorists quite well, and better than you'd expect GPT3.5 to. 

In [21]:
vectorstore.delete_collection()